In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1387198/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.7)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_5_49,0.990749,0.612913,0.986509,0.984636,0.986664,0.004730,0.229791,0.007944,0.005616,0.006910,0.023876,0.068779,1.007452,0.071193,140.707451,231.785281,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_5_48,0.990718,0.612950,0.986717,0.984998,0.986909,0.004746,0.229770,0.007822,0.005484,0.006783,0.023877,0.068894,1.007477,0.071312,140.700747,231.778577,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_5_47,0.990685,0.612981,0.986925,0.985361,0.987154,0.004763,0.229751,0.007699,0.005351,0.006656,0.023877,0.069017,1.007504,0.071439,140.693608,231.771438,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_5_46,0.990649,0.613006,0.987132,0.985724,0.987399,0.004782,0.229736,0.007577,0.005219,0.006529,0.023877,0.069148,1.007533,0.071575,140.686000,231.763830,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_5_45,0.990611,0.613024,0.987339,0.986087,0.987643,0.004801,0.229726,0.007455,0.005086,0.006402,0.023877,0.069288,1.007563,0.071720,140.677913,231.755743,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.913035,0.631520,0.614527,0.882867,0.817069,0.044468,0.218746,0.123227,0.024117,0.079178,0.114277,0.210875,1.070055,0.218276,136.225962,227.303792,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.912090,0.631347,0.614603,0.880450,0.816589,0.044951,0.218849,0.123203,0.024615,0.079386,0.115261,0.212017,1.070816,0.219459,136.204352,227.282181,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.911124,0.631160,0.614653,0.877980,0.816087,0.045446,0.218959,0.123187,0.025123,0.079603,0.116255,0.213180,1.071595,0.220662,136.182474,227.260304,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.910133,0.630956,0.614671,0.875459,0.815561,0.045952,0.219080,0.123181,0.025643,0.079831,0.117259,0.214364,1.072393,0.221888,136.160316,227.238146,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
